In [5]:
# Important Setup
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2
from import_src import import_src
import_src()

Append /home/ubuntu/tungpth/MLOpsVN/src to sys.path


In [2]:
import pandas as pd

In [37]:
from raw_data_processor import RawDataProcessor
import pickle
import json
from functools import partial

In [67]:
raw_df = pd.read_parquet('../../data/raw_data/phase-2/prob-2/raw_train.parquet').iloc[:1000]

In [68]:
with open('../../data/train_data/phase-2/prob-2/category_index.pickle', "rb") as f:
    category_index =  pickle.load(f)


In [69]:
# category_index

In [70]:
with open('../../data/train_data/phase-2/prob-2/features_config.json') as f:
    features_config = json.load(f)


In [71]:
categorical_cols = features_config['category_columns']

In [72]:
category_index

{'feature2': Index(['3pc', 'a/n', 'aes-sp3-d', 'any', 'argus', 'aris', 'arp', 'ax.25',
        'bbn-rcc', 'bna',
        ...
        'visa', 'vmtp', 'vrrp', 'wb-expak', 'wb-mon', 'wsn', 'xnet', 'xns-idp',
        'xtp', 'zero'],
       dtype='object', length=132),
 'feature3': Index(['-', 'dhcp', 'dns', 'ftp', 'ftp-data', 'http', 'irc', 'pop3', 'radius',
        'smtp', 'snmp', 'ssh', 'ssl'],
       dtype='object'),
 'feature4': Index(['ACC', 'CON', 'ECO', 'FIN', 'INT', 'REQ', 'RST'], dtype='object')}

In [73]:
apply_cat_func = lambda col, category_index_col: pd.Series(pd.Categorical(col, categories=category_index_col).codes)

In [74]:
dict_func = {key:partial(apply_cat_func, category_index_col=category_index[key]) for key in category_index.keys()}

In [75]:
dict_func

{'feature2': functools.partial(<function <lambda> at 0x7f4b662afe50>, category_index_col=Index(['3pc', 'a/n', 'aes-sp3-d', 'any', 'argus', 'aris', 'arp', 'ax.25',
        'bbn-rcc', 'bna',
        ...
        'visa', 'vmtp', 'vrrp', 'wb-expak', 'wb-mon', 'wsn', 'xnet', 'xns-idp',
        'xtp', 'zero'],
       dtype='object', length=132)),
 'feature3': functools.partial(<function <lambda> at 0x7f4b662afe50>, category_index_col=Index(['-', 'dhcp', 'dns', 'ftp', 'ftp-data', 'http', 'irc', 'pop3', 'radius',
        'smtp', 'snmp', 'ssh', 'ssl'],
       dtype='object')),
 'feature4': functools.partial(<function <lambda> at 0x7f4b662afe50>, category_index_col=Index(['ACC', 'CON', 'ECO', 'FIN', 'INT', 'REQ', 'RST'], dtype='object'))}

In [65]:
# dict_func = {key:'index' for key in category_index.keys()}
# dict_func

In [80]:
%%timeit

apply_df = raw_df.copy()
# for col in categorical_cols:
#     apply_df[col] = apply_df[col].astype("category")
#     apply_df[col] = pd.Categorical(
#         apply_df[col],
#         categories=category_index[col],
#     ).codes
apply_df[categorical_cols] = apply_df[categorical_cols].astype('category')

# apply_cat_func = lambda col: pd.Categorical(col, categories=category_index[col]).codes    
# def apply_cat_func(col):
#     # print(type(col))
#     print(col)
#     return pd.Categorical(col, categories=category_index[col]).codes
#     # return col

apply_df[categorical_cols] = apply_df[categorical_cols].transform(dict_func)



4.06 ms ± 7.43 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [83]:
%%timeit

apply_df = raw_df.copy()
for col in categorical_cols:
    # apply_df[col] = apply_df[col].astype("category")
    apply_df[col] = pd.Categorical(
        apply_df[col],
        categories=category_index[col],
    ).codes


1.05 ms ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
